# Realizar una recomendación para cada sitio 
## Para realizar Deploy en Stremlit

### Primero vamos a confeccionar un dataset con los datos de cada sitio con su respectiva recomendacion utilizando ML y luego realizar el deploy en streamlit

Importamos Bibliotecas

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

Cargamos los archivos ya procesado en un Dataframe

In [ ]:
# Cargar el DataFrame df_ml 
dl_ml = pd.read_csv('Ruta del archivo procesado')

Realizar transformaciones y agrupar datos por ciudad y estado

In [ ]:
# Obtener una lista de datos únicos de city por cada estado y la cantidad de id_sitios diferentes por ciudad
lista_ciudades_por_estado = df_ml.groupby(['state', 'city']).agg({'id_sitio': 'nunique'}).reset_index()

# Convertir 'id_sitio' a tipo entero
df_ml["id_sitio"] = df_ml["id_sitio"].astype(int)

# Remover caracteres no deseados de la columna 'text'
df_ml['text'] = df_ml['text'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))

# Definir df_stream si no está definido previamente
df_stream = pd.DataFrame()

Realizamos una funcion que usaremos para obtener la similitud

In [ ]:

# Función para obtener recomendaciones por título
def recomendacion(sitio, similitud, porciudad):
    indice = porciudad[porciudad["id_sitio"] == sitio].index
    if len(indice) > 0:
        indice = indice[0]
        if indice < len(similitud):
            distances = similitud[indice]
            lista_resto = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6] # modificando esta ultima parte podemos cambiar la cantidad de recomendaciones obtenidas
            recommended_titles = [porciudad.iloc[i]['id_sitio'] for i, _ in lista_resto]
            return recommended_titles
        else:
            return []
    else:
        return []


Iterando por cada ciudad de cada estado realizamos el proceso de recomendacion

In [ ]:

# Iterar sobre el DataFrame lista_ciudades_por_estado
for index, row in lista_ciudades_por_estado.iterrows():
    estado = row['state']
    ciudad = row['city']
    cantidad_id_sitio = row['id_sitio']
    
    print(f"Procesando estado: {estado}, ciudad: {ciudad}")
    
    # Filtrar el DataFrame df_ml por estado y ciudad
    porciudad = df_ml[(df_ml['state'] == estado) & (df_ml['city'] == ciudad)][['id_sitio', 'text']]
    porciudad["id_sitio"] = porciudad["id_sitio"].astype(int)
    
    # Tomar una muestra aleatoria del DataFrame porciudad si hay más de 1000 filas
    if len(porciudad) > 1000:
        porciudad_muestra = porciudad.sample(n=1000, random_state=42)
    else:
        porciudad_muestra = porciudad
    
    # Aplicar TF-IDF al texto en la muestra
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix_muestra = tfidf_vectorizer.fit_transform(porciudad_muestra['text'])
    
    # Calcular la similitud coseno entre vectores de texto en la muestra
    similitud_muestra = cosine_similarity(tfidf_matrix_muestra, tfidf_matrix_muestra)
    
    # Obtener recomendaciones y agregarlas al DataFrame
    porciudad_muestra['Recomendaciones'] = porciudad_muestra['id_sitio'].apply(lambda x: recomendacion(x, similitud_muestra, porciudad_muestra))
    
    # Agregar el resultado al DataFrame principal
    df_stream = pd.concat([df_stream, porciudad_muestra], ignore_index=True)



una vez obtenido el dataframe resultante lo almacenamos en un archivo para ser usado por la api

## El siguente es el codigo usado para realizar la Api utilizando Streamlit

se crea un entorno virtual para correr en forma local la api que luego va a ser exportada

todo el siguiente codigo se guarda en un archivo.py que va a ser el que va a correr en nuestra api

In [ ]:
import streamlit as st
import pandas as pd

# Insertar la imagen con el ancho calculado en píxeles
st.image("logo.png", width=200)

ruta_archivo ='Ruta del archivo'
df = pd.read_csv(ruta_archivo)
st.title('Busqueda y Recomendación')
# Definir el diccionario de mapeo
estados = {
    'TX': 'Texas',
    'CA': 'California',
    'PA': 'Pennsylvania',
    'NY': 'New York',
    'FL': 'Florida'
}
if st.checkbox('Buscar Restaurant '):
    # Obtener la selección del usuario
    opcion_abreviada = st.radio('Estado', ('CA','PA','FL','NY','TX',), format_func=lambda x: estados[x], horizontal=True)
    # Obtener el nombre completo del estado seleccionado
    nombre_completo = estados[opcion_abreviada]
    df_estado_seleccionado = df[df['state'] == opcion_abreviada]
    # Obtener la lista de ciudades únicas para el estado seleccionado
    ciudades_estado_seleccionado = df_estado_seleccionado['city'].unique().tolist()
    ciudade = st.selectbox('Elija ciudad', ciudades_estado_seleccionado)
    nombres = df[(df['state'] == opcion_abreviada) & (df['city'] == ciudade)]['name'].tolist()
    nombres_formateados = [f"{nombre} - {indice}" for indice, nombre in enumerate(nombres)]
    nombres_seleccionados = st.selectbox('Elija local', nombres_formateados)
    
    # Obtener el índice seleccionado
    indice_seleccionado = nombres_formateados.index(nombres_seleccionados)
    
    # Obtener los datos del restaurante seleccionado
    datos_seleccionados = df_estado_seleccionado[df_estado_seleccionado['name'] == nombres[indice_seleccionado]]
    
    # Crear un nuevo DataFrame con los datos seleccionados
    seleccion = pd.DataFrame(datos_seleccionados)
    nombre = seleccion['name'].iloc[0]
    direccion = seleccion['address'].iloc[0]
    catagoria = seleccion['category'].iloc[0]
    estrellas = str(seleccion['avg_rating'].iloc[0])
    web = seleccion['url'].iloc[0]
    
    col1,col2 = st.columns(2)
    with col1:
        st.map(seleccion)
    with col2:
        st.write('Nombre Local :', nombre)
        st.write('Direccion :', direccion)
        st.write('Categoria :', catagoria)
        st.write('Valoracion :', estrellas)
        st.write('web :', web)
    if st.checkbox('Ver Recomendacion '):
        seleccion_id = seleccion['recomendacion'].iloc[0] 
        # se crea un segundo dataFrame con el indice de la recomendacion
        recomendacion = df[df['id_sitio'] == seleccion_id] 

        nombrer = recomendacion['name'].iloc[0]
        direccionr = recomendacion['address'].iloc[0]
        catagoriar = recomendacion['category'].iloc[0]
        estrellasr = str(recomendacion['avg_rating'].iloc[0])
        webr = recomendacion['url'].iloc[0]
        col3,col4 = st.columns(2)
        with col3:
            st.write('Nombre Local :', nombrer)
            st.write('Direccion :', direccionr)
            st.write('Categoria :', catagoriar)
            st.write('Valoracion :', estrellasr)
            st.write('web :', webr)
        with col4:
            st.map(recomendacion.iloc[[0]])

if st.button('Busqueda Aleatoria'):
    # Selecciona un dato aleatorio del DataFrame
    dato = df.sample()    
    nombrea = dato['name'].iloc[0]
    direcciona = dato['address'].iloc[0]
    catagoriaa = dato['category'].iloc[0]
    estrellasa = str(dato['avg_rating'].iloc[0])
    weba = dato['url'].iloc[0]
    col5,col6 = st.columns(2)
    with col5:
        st.map(dato)
    with col6:
        st.write('Nombre Local :', nombrea)
        st.write('Direccion :', direcciona)
        st.write('Categoria :', catagoriaa)
        st.write('Valoracion :', estrellasa)
        st.write('web :', weba)

se realiza el archivo requerimientos.txt para poder hacer el deploy de la api